In [1]:
import numpy as np
import scipy.stats as st
import math
import matplotlib
from matplotlib import pyplot as plt

In [2]:
plt.rcParams.update({'legend.fontsize': 30, 'axes.labelsize' : 30, 'axes.titlesize': 20})

Приступим к практическому внедрению метода градиентного спуска в жизнь (и в нашу задачу, в частности).

In [3]:
N = 10**3
TernConst = 1

Тут представлен тернарный поиск с количеством шагов $TernConst$.

In [4]:
def _ternarnik(x, y, func):
    m1 = x + (y - x) / 3
    m2 = y - (y - x) / 3
#     print("m1 = {}, m2 = {}".format(m1, m2))
#     print("func(m1) = {}, func(m2) = {}".format(func(m1), func(m2)))
    if func(m1) < func(m2):
#         print("x = {}, y = {}".format(m1, y))
        return (m1, y)
    else:
#         print("x = {}, y = {}".format(x, m2))
        return (x, m2)
def do_ternar(x, y, func):
    for i in range(TernConst):
        (x, y) = _ternarnik(x, y, func)
    return (x, y)

Эта операция служит для получения случайных значений массивов $p$ и $\pi$. Получившиеся значения $p \in [0, 2]$.

In [5]:
def make_random(n):
    P_base = list(np.random.rand(n - 1))
    P_base.append(0)
    P_base.append(1)
    P_base.sort()
    P = np.array([P_base[i] - P_base[i - 1] for i in range(1, n + 1)])
    P = np.floor(P * 100) / 100
    P[0] += 1 - np.sum(P)
    p = np.floor(np.random.rand(n, n) * 2 * 100) / 100
    #p = (np.random.rand(n, n) - np.ones((n, n)) / 2) * 2
    return [P, p]

Здесь описаны функции получения значений $F$ и $\nabla F$ (по уже полученной формуле и беря проекцию под конец).

In [6]:
def base_func(P, p, x):
    x = np.array(x)
    p = np.array(p)
    result = 0
    for (Pi, pi) in zip (P, p):
#         print(Pi)
#         print(x)
#         print(pi)
#         print(pi @ x)
        result += Pi * math.log(pi @ x)
    return result
def base_grad(P, p, x):
    n = len(x)
    x = np.array(x)
    p = np.array(p)
    result = np.zeros(n)
    for (Pi, pi, p_j) in zip (P, p, p.transpose()):
        result += pi * (Pi / (pi @ x))
#     print(result)
    result -= np.ones((n)) * ((np.ones((n)) @ result) / n)
    return result

In [7]:
def make_func(P, p):
    return lambda x: base_func(P, p, x)
def make_grad(P, p):
    return lambda x: base_grad(P, p, x)

Следующая точка градиентного поиска получается из этих функций. Первую из них, с заданным шагом, мы пока не используем. Во второй, мы сначала вычисляем предельную точку множества по направлению градиента (которая выйдет за $x \succeq 0$). А затем отправляем $x_{i - 1}$ и $x_{пред}$ на тернарный поиск и возвращаем среднее от результата.

In [8]:
def next_grad_point_with_step(x_prev, lambd, grad):
    return(x_prev + lambd * grad(x_prev))
def next_grad_point(x_prev, grad, func):
    grad = grad(x_prev)
#     print(grad)
    t = None
    for (x, g) in zip(x_prev, grad):
        comp = None
        if g < 0:
            comp = -x / g
#         print('x = {}, grad = {}, t = {}'.format(x, g, comp))
            if (t == None) or comp < t:
                t = comp
#     print(x_prev + t * 98 / 100 * grad)
    points = do_ternar(x_prev, x_prev + t * 98 / 100 * grad, func)
    return((points[0] + points[1]) / 2)

Конфиг наших экспериментов. Состоит из задаваемой части и случайной, в которой мы только задаем размерность.

In [9]:
config_base = [
    {
        'P': [0.5, 0.5],
        'p': [[1, 1], [1, 1]], 
        'lambda': 1,
        'Graphs': []
    },
    {
        'P': [0.5, 0.5],
        'p': [[1, 20], [1, 1]],
        'lambda': 1,
        'Graphs': []
    },
    {
        'P': [0.5, 0.5],
        'p': [[19, 20], [1, 1]], 
        'lambda': 1,
        'Graphs': []
    },
    {
        'P': [0.1, 0.9],
        'p': [[19, 1], [20, 1]], 
        'lambda': 1,
        'Graphs': []
    }
]
random_config = [3, 3, 3, 4, 4, 4, 4, 6, 6, 6, 10, 10, 20]

In [10]:
data = config_base.copy()
for k in random_config:
    ran = make_random(k)
    data.append({
        'P': ran[0],
        'p': ran[1],
        'lambda': 1,
        'Graphs': []
    })

Следующии три окошка представляют из себя три серии экспериментов, с различным выбором начальных точек:
  * $x_0 = (\frac{1}{n}, \frac{1}{n},..., \frac{1}{n})$
  * $x_0 = (1, 0,..., 0)$
  * $x_0 = (\frac{1}{2}, \frac{1}{2}, 0,..., 0)$
  Каждая серия заканчивается либо при уменьшении нормы градиента или нормы разности $x_i - x_{i - 1}$ ниже определенного значения, либо по достижении предельного количества шагов.

In [11]:
for data_map in data:
    P = data_map['P']
    p = data_map['p']
    lambd = data_map['lambda'] / 10
    func = make_func(P, p)
    grad = make_grad(P, p)
    x0 = np.ones((len(P))) / len(P)
    values = [func(x0)]
    Xs = [x0]
    data_map['Graphs'].append({
        'x0': x0,
        'values': values,
        'Xs': Xs
    })
    print(data_map)
    for i in range(N):
#         print("Grad: {}".format(grad(Xs[-1])))
        if np.linalg.norm(grad(Xs[-1])) < 0.01:
            print("Check! Grad\n")
            break
        Xs.append(next_grad_point(Xs[-1], grad, func))
#         print("Next x: {}".format(Xs[-1]))
        values.append(func(Xs[-1]))
        if (np.linalg.norm(Xs[-1] - Xs[-2]) < 10**-4):
            print("Check! Norm\n")
            break
    if i == N - 1:
        print("Check! Ammount")

{'P': [0.5, 0.5], 'p': [[1, 1], [1, 1]], 'lambda': 1, 'Graphs': [{'x0': array([0.5, 0.5]), 'values': [0.0], 'Xs': [array([0.5, 0.5])]}]}
Check! Grad

{'P': [0.5, 0.5], 'p': [[1, 20], [1, 1]], 'lambda': 1, 'Graphs': [{'x0': array([0.5, 0.5]), 'values': [1.1756876285817388], 'Xs': [array([0.5, 0.5])]}]}
Check! Norm

{'P': [0.5, 0.5], 'p': [[19, 20], [1, 1]], 'lambda': 1, 'Graphs': [{'x0': array([0.5, 0.5]), 'values': [1.4852072327848505], 'Xs': [array([0.5, 0.5])]}]}
Check! Norm

{'P': [0.1, 0.9], 'p': [[19, 1], [20, 1]], 'lambda': 1, 'Graphs': [{'x0': array([0.5, 0.5]), 'values': [2.3464962407465344], 'Xs': [array([0.5, 0.5])]}]}
Check! Norm

{'P': array([0.5 , 0.44, 0.06]), 'p': array([[0.44, 0.66, 1.32],
       [1.04, 0.5 , 1.52],
       [0.35, 1.45, 0.59]]), 'lambda': 1, 'Graphs': [{'x0': array([0.33333333, 0.33333333, 0.33333333]), 'values': [-0.11234835360291968], 'Xs': [array([0.33333333, 0.33333333, 0.33333333])]}]}
Check! Norm

{'P': array([0.42, 0.16, 0.42]), 'p': array([[1.43,

In [12]:
for data_map in data:
    P = data_map['P']
    p = data_map['p']
    lambd = data_map['lambda'] / 10
    func = make_func(P, p)
    grad = make_grad(P, p) 
    x0 = np.zeros(len(P))
    x0[0] = 0.99
    x0[1] = 0.01
    values = [func(x0)]
    Xs = [x0]
    data_map['Graphs'].append({
        'x0': x0,
        'values': values,
        'Xs': Xs
    })
    print(data_map)
    for i in range(N):
#         print("Grad: {}".format(grad(Xs[-1])))
        if np.linalg.norm(grad(Xs[-1])) < 0.01:
            print("Check! Grad\n")
            break
        Xs.append(next_grad_point(Xs[-1], grad, func))
#         print("Next x: {}".format(Xs[-1]))
        values.append(func(Xs[-1]))
        if (np.linalg.norm(Xs[-1] - Xs[-2]) < 10**-4):
            print("Check! Norm\n")
            break
    if i == N - 1:
        print("Check! Ammount")

{'P': [0.5, 0.5], 'p': [[1, 1], [1, 1]], 'lambda': 1, 'Graphs': [{'x0': array([0.5, 0.5]), 'values': [0.0], 'Xs': [array([0.5, 0.5])]}, {'x0': array([0.99, 0.01]), 'values': [0.0], 'Xs': [array([0.99, 0.01])]}]}
Check! Grad

{'P': [0.5, 0.5], 'p': [[1, 20], [1, 1]], 'lambda': 1, 'Graphs': [{'x0': array([0.5, 0.5]), 'values': [1.1756876285817388, 1.407903920679879, 1.4684768622370055, 1.4878722847883117, 1.4944241759814736, 1.4966756050444372, 1.497453740049126, 1.4977232110939578, 1.4978165938333992, 1.4978489624457942], 'Xs': [array([0.5, 0.5]), array([0.17333333, 0.82666667]), array([0.06008889, 0.93991111]), array([0.02083081, 0.97916919]), array([0.00722135, 0.99277865]), array([0.0025034, 0.9974966]), array([8.67845692e-04, 9.99132154e-01]), array([3.00853173e-04, 9.99699147e-01]), array([1.04295767e-04, 9.99895704e-01]), array([3.61558658e-05, 9.99963844e-01])]}, {'x0': array([0.99, 0.01]), 'values': [0.08697665356171899], 'Xs': [array([0.99, 0.01])]}]}
Check! Norm

{'P': [0.5, 0

{'P': array([0.15, 0.05, 0.18, 0.  , 0.04, 0.04, 0.05, 0.08, 0.  , 0.03, 0.01,
       0.01, 0.03, 0.  , 0.07, 0.07, 0.  , 0.13, 0.02, 0.04]), 'p': array([[1.44, 0.99, 1.41, 0.75, 1.53, 0.13, 1.22, 1.71, 1.96, 1.32, 1.17,
        0.74, 0.21, 0.46, 1.57, 0.32, 1.9 , 1.59, 1.96, 1.41],
       [0.3 , 0.43, 0.51, 0.04, 0.1 , 1.59, 1.5 , 0.59, 1.39, 0.69, 1.94,
        0.32, 1.84, 0.28, 1.4 , 0.56, 0.88, 1.58, 0.46, 0.14],
       [1.59, 1.31, 1.83, 1.02, 0.4 , 0.56, 0.6 , 1.87, 0.53, 0.49, 0.3 ,
        1.17, 1.23, 1.91, 1.88, 1.46, 0.6 , 1.96, 1.06, 0.27],
       [0.7 , 1.98, 0.94, 1.08, 0.35, 1.89, 1.71, 1.1 , 0.82, 1.15, 0.14,
        1.7 , 1.62, 1.89, 1.05, 0.73, 0.35, 0.95, 1.59, 1.19],
       [0.83, 0.21, 1.54, 1.45, 0.71, 0.49, 1.61, 1.89, 0.25, 1.31, 0.83,
        0.56, 1.07, 0.23, 0.2 , 1.25, 0.66, 0.63, 1.64, 0.47],
       [1.84, 0.3 , 1.89, 0.89, 0.23, 1.43, 1.27, 1.96, 0.95, 1.08, 0.08,
        1.74, 0.04, 1.68, 0.93, 0.46, 1.7 , 1.5 , 1.4 , 0.95],
       [1.25, 0.09, 1.56, 0.93,

In [13]:
for data_map in data:
    P = data_map['P']
    p = data_map['p']
    lambd = data_map['lambda'] / 10
    func = make_func(P, p)
    grad = make_grad(P, p) 
    x0 = np.zeros(len(P))
    x0[0] = 0.5
    x0[1] = 0.5
    values = [func(x0)]
    Xs = [x0]
    data_map['Graphs'].append({
        'x0': x0,
        'values': values,
        'Xs': Xs
    })
    print(data_map)
    for i in range(N):
#         print("Grad: {}".format(grad(Xs[-1])))
        if np.linalg.norm(grad(Xs[-1])) < 0.01:
            print("Check! Grad\n")
            break
        Xs.append(next_grad_point(Xs[-1], grad, func))
#         print("Next x: {}".format(Xs[-1]))
        values.append(func(Xs[-1]))
        if (np.linalg.norm(Xs[-1] - Xs[-2]) < 10**-4):
            print("Check! Norm\n")
            break
    if i == N - 1:
        print("Check! Ammount")

{'P': [0.5, 0.5], 'p': [[1, 1], [1, 1]], 'lambda': 1, 'Graphs': [{'x0': array([0.5, 0.5]), 'values': [0.0], 'Xs': [array([0.5, 0.5])]}, {'x0': array([0.99, 0.01]), 'values': [0.0], 'Xs': [array([0.99, 0.01])]}, {'x0': array([0.5, 0.5]), 'values': [0.0], 'Xs': [array([0.5, 0.5])]}]}
Check! Grad

{'P': [0.5, 0.5], 'p': [[1, 20], [1, 1]], 'lambda': 1, 'Graphs': [{'x0': array([0.5, 0.5]), 'values': [1.1756876285817388, 1.407903920679879, 1.4684768622370055, 1.4878722847883117, 1.4944241759814736, 1.4966756050444372, 1.497453740049126, 1.4977232110939578, 1.4978165938333992, 1.4978489624457942], 'Xs': [array([0.5, 0.5]), array([0.17333333, 0.82666667]), array([0.06008889, 0.93991111]), array([0.02083081, 0.97916919]), array([0.00722135, 0.99277865]), array([0.0025034, 0.9974966]), array([8.67845692e-04, 9.99132154e-01]), array([3.00853173e-04, 9.99699147e-01]), array([1.04295767e-04, 9.99895704e-01]), array([3.61558658e-05, 9.99963844e-01])]}, {'x0': array([0.99, 0.01]), 'values': [0.086976

       [0.57, 1.02, 0.64, 1.37]]), 'lambda': 1, 'Graphs': [{'x0': array([0.25, 0.25, 0.25, 0.25]), 'values': [-0.19777802131211633, 0.037978596233609635, 0.09364068387606744, 0.11036679737760048, 0.11586696992010101, 0.1177382554107201, 0.11838272523700995, 0.11860563229683461, 0.11868284558562699, 0.1187096055103997], 'Xs': [array([0.25, 0.25, 0.25, 0.25]), array([0.08666667, 0.3116157 , 0.12580326, 0.47591437]), array([0.03004444, 0.3286063 , 0.09244147, 0.54890778]), array([0.01041541, 0.33393509, 0.08246339, 0.57318611]), array([0.00361067, 0.33570787, 0.07922859, 0.58145286]), array([0.0012517 , 0.33631311, 0.07813579, 0.58429939]), array([4.33922846e-04, 3.36521792e-01, 7.77604656e-02, 5.85283820e-01]), array([1.50426587e-04, 3.36593996e-01, 7.76307764e-02, 5.85624801e-01]), array([5.21478833e-05, 3.36619011e-01, 7.75858687e-02, 5.85742973e-01]), array([1.80779329e-05, 3.36627680e-01, 7.75703068e-02, 5.85783935e-01])]}, {'x0': array([0.99, 0.01, 0.  , 0.  ]), 'values': [-1.023426

Наконец, сравнительные графики для каждого набора $p$ и $\pi$ для всех трех серий

In [14]:
rows_per_data = 2 + (len(data[0]['Graphs']) // 3 + 1) * 2 + 1
gridsize = (len(data) * rows_per_data, 3)
fig = plt.figure(figsize=(40, 40 * len(data)), facecolor=matplotlib.colors.get_named_colors_mapping()['xkcd:cream'])
plt.subplots_adjust(hspace=0.8)
for i in range(len(data)):
    data_map = data[i]
    P = data_map['P']
    p = data_map['p']
    labels = ['$x_{' + '{}'.format(i + 1) + '}$' for i in range(len(P))]
    ax0 = plt.subplot2grid(gridsize, (i * rows_per_data, 0), colspan=3, rowspan=2)
    params = ''
    for (Pi, pi) in zip(P, p):
        params += r'$\pi$: {:.2f}, $p_j$: {}'.format(Pi, pi) + '\n'
    ax0.set_title(params, {'fontsize': 30}, loc='left')
    ax0.set_xlabel(r'$k$')
    ax0.set_ylabel(r'$F(x_k)$')
    graphs = data_map['Graphs']
    for j in range(len(graphs)):
        graph = graphs[j]
        Xs = np.array(graph['Xs'])
        x0 = graph['x0']
        n = len(Xs)
        graph_x = range(n)
        ax0.plot(graph_x, graph['values'], label='$x_0 = ${}'.format(np.floor(x0*100) / 100))
        axi = plt.subplot2grid(gridsize, (i * rows_per_data + (j // 3) * 2 + 2, j % 3), colspan=1, rowspan=2)
        params = ''
        axi.stackplot(graph_x, *[Xs[:, i] for i in range(len(P))], labels=labels)
        axi.legend()
        axi.set_title('$x_0 = ${}'.format(np.floor(x0*100) / 100))
        axi.set_xlabel(r'$k$')
        axi.set_ylabel(r'$X$')
    ax0.legend()

In [15]:
import pprint
pprint.pprint(data)

[{'Graphs': [{'Xs': [array([0.5, 0.5])],
              'values': [0.0],
              'x0': array([0.5, 0.5])},
             {'Xs': [array([0.99, 0.01])],
              'values': [0.0],
              'x0': array([0.99, 0.01])},
             {'Xs': [array([0.5, 0.5])],
              'values': [0.0],
              'x0': array([0.5, 0.5])}],
  'P': [0.5, 0.5],
  'lambda': 1,
  'p': [[1, 1], [1, 1]]},
 {'Graphs': [{'Xs': [array([0.5, 0.5]),
                     array([0.17333333, 0.82666667]),
                     array([0.06008889, 0.93991111]),
                     array([0.02083081, 0.97916919]),
                     array([0.00722135, 0.99277865]),
                     array([0.0025034, 0.9974966]),
                     array([8.67845692e-04, 9.99132154e-01]),
                     array([3.00853173e-04, 9.99699147e-01]),
                     array([1.04295767e-04, 9.99895704e-01]),
                     array([3.61558658e-05, 9.99963844e-01])],
              'values': [1.175687628581738

                     array([0.00166893, 0.58393758, 0.41439349]),
                     array([5.78563794e-04, 5.84467686e-01, 4.14953751e-01]),
                     array([2.00568782e-04, 5.84651237e-01, 4.15148194e-01]),
                     array([6.95305111e-05, 5.84714842e-01, 4.15215627e-01]),
                     array([2.41039105e-05, 5.84736889e-01, 4.15239007e-01])],
              'values': [0.2515136410989708,
                         0.31317202919646536,
                         0.33013228656931215,
                         0.3358350738286503,
                         0.3377950918198992,
                         0.3384726617177448,
                         0.3387073289984885,
                         0.33878865365387667,
                         0.3388168430045627,
                         0.33882661492900157],
              'x0': array([0.33333333, 0.33333333, 0.33333333])},
             {'Xs': [array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ])],
              'values': 

                         -0.18236492729787926,
                         -0.1707315219070054,
                         -0.16674692226291346,
                         -0.16537124629721794,
                         -0.16489501847911348,
                         -0.1647300068235748,
                         -0.16467281246567977,
                         -0.16465298625149605],
              'x0': array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667])},
             {'Xs': [array([0.99, 0.01, 0.  , 0.  , 0.  , 0.  ]),
                     array([0.99, 0.01, 0.  , 0.  , 0.  , 0.  ])],
              'values': [-0.6514428372039373, -0.6514428372039373],
              'x0': array([0.99, 0.01, 0.  , 0.  , 0.  , 0.  ])},
             {'Xs': [array([0.5, 0.5, 0. , 0. , 0. , 0. ]),
                     array([0.5, 0.5, 0. , 0. , 0. , 0. ])],
              'values': [-0.4774279561203905, -0.4774279561203905],
              'x0': array([0.5, 0.5, 0. , 0. , 0. , 0. ])}],